In [ ]:
import numpy as np
import glob
import os
from models.JetPointNet import PointNetSegmentation


def load_data_from_npz(npz_file):
    data = np.load(npz_file)
    feats = data['feats']  # Shape: (num_samples, 859, 6)
    frac_labels = data['frac_labels']  # Shape: (num_samples, 859)
    tot_labels = data['tot_labels']  # Shape: (num_samples, 859)
    tot_truth_e = data['tot_truth_e']  # Shape: (num_samples, 859) (This is the true total energy deposited by particles into this cell)
    return feats, frac_labels, tot_labels, tot_truth_e

# Setup
os.environ['CUDA_VISIBLE_DEVICES'] = ""  # Disable GPU
abs_model_path = "saved_model/PointNetModel.keras"

TEST_DIR = '/data/mjovanovic/jets/processed_files/2000_events_w_fixed_hits/SavedNpz/test'

abs_model = PointNetSegmentation(num_points=859, num_classes=1)
abs_model.load_weights(abs_model_path)

npz_files = glob.glob(os.path.join(TEST_DIR, '*.npz'))



In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from models.JetPointNet import PointNetSegmentation

# Define the path to your model's weights
abs_model_path = "saved_model/PointNetModel.keras"

# Initialize your model
abs_model = PointNetSegmentation(num_points=859, num_classes=1)

# Load the weights into the model
abs_model.load_weights(abs_model_path)

# Save the model plot to a file and then display it
plot_image_path = 'model_structure.png'
plot_model(abs_model, to_file=plot_image_path, show_shapes=True, show_layer_names=True)

# To display the image, you would normally use a library like matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread(plot_image_path)
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.axis('off')  # Turn off axis numbers and ticks
plt.show()

In [ ]:
# Initialize metrics
frac_mae_metrics = []
abs_mae_metrics = []

# Process each file
for npz_file_idx, npz_file in enumerate(npz_files):
    feats, frac_labels, tot_labels, tot_truth_e = load_data_from_npz(npz_file)
    current_energies = feats[:, :, 4]  # Energy is at index 4

    # Predictions
    abs_predicted = np.squeeze(abs_model.predict(feats), axis=-1)

    # Compute predicted energies
    abs_predicted_energies = abs_predicted  # Directly the predicted energies

    for i in range(len(tot_labels)):  # Iterate over samples
        valid_indices = tot_labels[i] != -1  # Indices where labels are not -1
        
        # Compute Mean Absolute Error (MAE) for energy reconstruction, excluding -1 labels
        abs_mape = np.sum(abs_predicted_energies[i][valid_indices])/np.sum(tot_labels[i][valid_indices])

        abs_mae_metrics.append(abs_mape)

    # Optionally, print comparisons for the first few samples of the first few files
    if npz_file_idx < 3:  # Print for the first 3 files
        print(f"File: {npz_file}")
        for sample_idx in range(min(5, len(feats))):  # Print first 3 samples
            valid_indices = tot_labels[sample_idx] != -1
            print(f"\nSample {sample_idx}:")

            abs_predicted_formatted = [f'{value:.5f}' for value in abs_predicted_energies[sample_idx][valid_indices]]
            true_energies_formatted = [f'{value:.5f}' for value in tot_labels[sample_idx][valid_indices]]

            print(f"Absolutely Predicted Energies: \n[{', '.join(abs_predicted_formatted)}]")  # Join the formatted strings for printing
            print(f"True Energies: \n[{', '.join(true_energies_formatted)}]\n")

# Compute and print overall metrics
average_abs_mape = np.mean(abs_mae_metrics)

print(f"Average MAPE for Absolute Predictions (excluding -1 labels): {(average_abs_mape)*100}%")